<a href="https://colab.research.google.com/github/Winzen/mides-rascunho/blob/main/code/scraping/go/%5Bgo%5Dextrair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Verificar IP

In [ ]:
import requests

my_country = requests.get("https://api.myip.com/")
my_country.json()

{'ip': '35.243.145.119', 'country': 'United States', 'cc': 'US'}

# Importação

In [241]:
import requests
import pandas as pd
import os
import concurrent.futures
from time import sleep
from urllib3 import PoolManager
import glob
import shutil

class Dados:

  def __init__(self, municipio: str, ano: int,
               dados: list, colunas: list) -> None:
    self.municipio = municipio
    self.ano = str(ano)
    self.dados = dados
    self.colunas = colunas

def get_municipios() -> list:
  # Define the URL you want to send a POST request to
  url = 'https://www.tcmgo.tc.br/pentaho/plugin/cda/api/doQuery'

  # Define the data you want to send in the POST request
  data = {
  "path":	"/system/cidadao/dashboards/Despesas.cda",
  "dataAccessId":	"mdxMunicipio",
  "outputIndexId":	1,
  "pageSize":	0,
  "pageStart":	0,
  "sortBy":	"",
  "paramsearchBox":	""}

  # Make the POST request
  response = requests.post(url, data=data)

  municipio_json = response.json()

  municipios = [row[0] for row in municipio_json["resultset"]]
  return municipios


def get_data_from_munipio_ano(municipio: str, ano: int):
  # Define the URL you want to send a POST request to
  url = 'https://www.tcmgo.tc.br/pentaho/plugin/cda/api/doQuery'

  # Define the data you want to send in the POST request
  data = {"paramparamMunicipio":	municipio,
  "paramparamAno":	ano,
  "path":	"/system/cidadao/dashboards/Despesas.cda",
  "dataAccessId":	"sqlEmpenhos",
  "outputIndexId":	1,
  "pageSize":	200,
  "pageStart":	0,
  "paramsearchBox":	""}

  # Make the POST request
  response = http.request('POST', url, fields=data, encode_multipart=False).json()
  dados = response["resultset"]
  colunas = [column["colName"] for column in response['metadata']]

  return Dados(municipio, ano, dados, colunas)


def save_dados(dados: object) -> None:

  path_input = os.path.join(os.getcwd(), "input", dados.ano, dados.municipio)
  df = pd.DataFrame(dados.dados, columns=dados.colunas)
  os.makedirs(path_input, exist_ok=True)
  df.to_csv(os.path.join(path_input, f"empenhos_{dados.ano}.csv"), index=False)

def send_folder_drive(path_drive: str, path_input: str) -> None:

  if not len(os.listdir(path_input)) > 0:
    raise Exception("Essa Pasta vazia")
    return None

  shutil.make_archive(path_drive,
                      'zip',
                      path_input)

def df_from_all_csv(csvs: list) -> pd.DataFrame:

  df = pd.concat([pd.read_csv(csv) for csv in csvs])
  # df = df.drop_duplicates()

  return df

# Pegar Dados Municipio

In [242]:
list_municipios = get_municipios()
anos = list(range(2019, 2025))
headers = {
     "Content-Type": "application/x-www-form-urlencoded"
}
http = PoolManager(maxsize=10, headers=headers)

for municipio in list_municipios:

  with concurrent.futures.ThreadPoolExecutor() as executor:
      datas = [dados for dados in executor.map(
          lambda ano: get_data_from_munipio_ano(municipio, ano), anos)]
  list(map(save_dados, datas))


# Analise

## Numero de arquivos por ano

In [253]:
for ano in anos:
  path_verify = f"/content/input/{ano}/*/*.csv"
  arquivos = glob.glob(path_verify)
  print(f"Ano de {ano} tem {len(arquivos)} arquivos")

Ano de 2019 tem 246 arquivos
Ano de 2020 tem 246 arquivos
Ano de 2021 tem 246 arquivos
Ano de 2022 tem 246 arquivos
Ano de 2023 tem 246 arquivos
Ano de 2024 tem 246 arquivos


## Formar DataFrame

In [248]:
path_verify = f"/content/input/**/**/*.csv"
csv_files_verify = glob.glob(path_verify)
df = df_from_all_csv(csv_files_verify)
df

# Mandar para o drive Manualmente

In [ ]:
path_drive_output = f"/content/drive/MyDrive/DataBase/world_bank/go/empenhos_go"
send_folder_drive(path_drive_output,
                "/content/input/")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
